This notebook is to: 
- scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
- create a dataframe accordingly

In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd 

In [2]:
# get html from wiki page and create soup object, using lxml parser
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url)
soup = BeautifulSoup(source.text, 'lxml')

#using soup object to get the data
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())

    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
df_canada = pd.DataFrame(data = data,columns = columns)
df_canada.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
len(df_canada['Postal code'].unique())

180

In [4]:
# print the original shape of the dataset
df_canada.shape

(180, 3)

In [5]:
# drop Borough=='Not assigned'
df_canada = df_canada[df_canada.Borough!='Not assigned']
# print the shape of the dataset ignore Borough == 'Not assigned'
df_canada.shape

(103, 3)

In [6]:
# check the unique Postal codes
len(df_canada['Postal code'].unique())

103

In [7]:
# check the Neighborhood == 'Not assigned'
df_canada[df_canada.Neighborhood=='Not assigned'].count()

Postal code     0
Borough         0
Neighborhood    0
dtype: int64

In [8]:
# take a look at the head
df_canada.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


It turns out the Neighborhood does not have values with 'Not assigned', and there is no duplicated Postal codes in the dataset,
in stead, there are values in Neigborgood seperated with '/', while it is required to be ',', therefore it has to be replaced with ','.

In [9]:
def replace_function(x):
    temp_list = x.split('/')
    out = [item.strip() for item in temp_list]
    return ', '.join(out)

In [10]:
df_canada.Neighborhood = df_canada.Neighborhood.apply(replace_function)

In [11]:
# reset index
df_canada.reset_index(inplace=True)
df_canada.drop(columns=['index'], inplace=True)

In [12]:
df_canada.columns=['PostalCode', 'Borough', 'Neighborhood']

In [13]:
# take a look at the head
df_canada.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
# print out shape
df_canada.shape

(103, 3)